In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

import numpy as np

import pandas as pd

from scipy import linalg
from scipy.spatial import Voronoi, voronoi_plot_2d

from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

In [ ]:
flags = pd.read_feather('../data/all_flags.feather')
tf_idf = pd.read_feather('../data/flag_description_tf_idf.feather')

In [ ]:
flags

In [ ]:
data = pd.DataFrame({'flag-country': flags.apply(lambda r: f'{r['Flag(s)']}-{r['State']}', axis=1)}).join(tf_idf)
data

In [ ]:
X = tf_idf
Y = data['flag-country']

In [ ]:

pca = PCA().fit(X)

In [ ]:
fig, ax = plt.subplots()

cum_var = np.sqrt(np.cumsum(pca.singular_values_**2) / np.sum(pca.singular_values_**2))
ax.plot(cum_var)

plt.show()

In [ ]:
all_var = np.searchsorted(cum_var, 0.99)
all_var

In [ ]:
pca.n_components_

In [ ]:
all_var = 100

In [ ]:
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import metrics
from tempfile import mkdtemp

cachedir = mkdtemp()
categorizers = list()

for pca_modes in [2, 3, 5, 10, 20, 50, 100, all_var]:
    for k in range(3, 31):
        pipe = make_pipeline(
            PCA(n_components=pca_modes, random_state=932),
            KMeans(n_clusters=k, random_state=42),
            memory=cachedir
        )
        
        categorizers.append(pipe)

model_metrics = dict()
for cat in categorizers:
    cat.fit(X, Y)
    labels = cat.predict(X)
    categories = {label: Y[labels == label].values for label in np.unique(labels)}
    avg = np.average(list(map(len, categories.values())))
    # var = np.average((np.fromiter(map(len, categories.values()), dtype=int) - avg)**2)
    silhouette_score = metrics.silhouette_score(X, cat.get_params()['kmeans'].labels_, metric='euclidean')
    model_metrics[cat] = silhouette_score

best = pipe = max(categorizers, key=model_metrics.get)
print(best.get_params()['pca__n_components'], best.get_params()['kmeans__n_clusters'], model_metrics[best])

categories = {label: Y[labels == label].values for label in np.unique(labels)}

In [ ]:
# for model, metric in model_metrics.items():
#     print(f'k={model.get_params()['kmeans__n_clusters']}\t: {metric}')

In [ ]:
# Handpicked model

best = pipe = make_pipeline(
    PCA(n_components=2),
    KMeans(n_clusters=10)
)

best.fit(X)

In [ ]:
centers_2d = pipe.named_steps['kmeans'].cluster_centers_[:, :2]

In [ ]:

fig, ax = plt.subplots(figsize=(10,10))

pts = (X @ pca.components_[:2].T).T.values
# plt.scatter(*pts)

for i, (x, y) in enumerate(pts.T):
    image = plt.imread(flags['path'][i])
    offset_img = OffsetImage(image, zoom=0.1)
    abb = AnnotationBbox(offset_img, (x, y), xycoords='data', frameon=False, zorder=0)
    ax.add_artist(abb)

ax.update_datalim(pts.T)
ax.autoscale()

xlim = ax.get_xlim()
ylim = ax.get_ylim()

# plt.scatter(*centers_2d.T, zorder=20, color='r', alpha=0.5)
voronoi_plot_2d(Voronoi(centers_2d), ax=ax)

ax.set_xlim(xlim)
ax.set_ylim(ylim)

ax.set_title('Flags on the first two PCs')
ax.set_xlabel('PC 1')
ax.set_ylabel('PC 2')

ax.set_facecolor('lightgray')
fig.savefig('../writeup/res/flag-pca.pdf')

plt.show()

In [ ]:
# want to analyze the principal components for patterns

components = pca.inverse_transform(pca.components_[:3] @ X.T)
components.columns = tf_idf.columns

for idx, component in components.iterrows():
    sorted_words = sorted(components.columns, key=lambda c: component[c])
    
    print(f'{idx}: {sorted_words[:3]} vs {sorted_words[-3:]}')


"elliptical" only shows up in the description for Costa Rica, and two of these words are conjunctions, this may not actually be that useful.

In [ ]:
# Let's try to see what the "average" flag per category is described as
# I'm not sure how to 'undo' tf-idf, but luckily sklearn has a TfidfVectorizer, which I will hopefully implement soon...

avg_flags = pd.DataFrame(pipe.named_steps['pca'].inverse_transform(pipe.named_steps['kmeans'].cluster_centers_), columns=pipe.feature_names_in_)
avg_flags

In [ ]:
# This runs really slow for some reason

fig, ax = plt.subplots()

for label, group in categories.items():
    for i, flag in enumerate(group):
        image = plt.imread(flags[data['flag-country'] == flag]['path'].values[0])
        offset_img = OffsetImage(image, zoom=0.1)
        abb = AnnotationBbox(offset_img, (label, i))
        ax.add_artist(abb)

ax.autoscale()

plt.show()

# GMMs

In [ ]:

gmm_models = list()
gmm_metrics = dict()

gmm_cache = mkdtemp()

for pc_modes in [2, 3, 5, 10, 20, 50, 100, all_var]:
    for components in range(2, 30):
        gmm = make_pipeline(
            PCA(n_components=pc_modes),
            GaussianMixture(n_components=5),
            memory=gmm_cache
        )
        gmm_models.append(gmm)

        gmm.fit(X)
        gmm_metrics[gmm] = metrics.silhouette_score(X, gmm.predict(X), metric='euclidean')

best_gmm = max(gmm_models, key=gmm_metrics.get)
print(f'Best GMM: {best_gmm.get_params()['pca__n_components']}, {best_gmm.get_params()['gaussianmixture__n_components']}: {gmm_metrics[best_gmm]}')

In [ ]:
best_gmm = make_pipeline(
    PCA(n_components=2),
    GaussianMixture(n_components=5),
    memory=gmm_cache
)

best_gmm.fit(X)

metrics.silhouette_score(X, best_gmm.predict(X), metric='euclidean')

In [ ]:
import pickle

with open('./best_gmm.pkl', mode='wb') as f:
    pickle.dump(best_gmm, f)

# with open('./best_gmm.pkl', mode='rb') as f:
#     best_gmm = pickle.load(f)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

pcs = [0, 1]
pts = (X @ pca.components_[pcs].T).T.values
# plt.scatter(*pts)

for i, (x, y) in enumerate(pts.T):
    image = plt.imread(flags['path'][i])
    offset_img = OffsetImage(image, zoom=0.1)
    abb = AnnotationBbox(offset_img, (x, y), xycoords='data', frameon=False, zorder=0)
    ax.add_artist(abb)

ax.update_datalim(pts.T)
ax.autoscale()

means = best_gmm.get_params()['gaussianmixture'].means_
covariances = best_gmm.get_params()['gaussianmixture'].covariances_
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# https://scikit-learn.org/stable/auto_examples/mixture/plot_gmm.html#sphx-glr-auto-examples-mixture-plot-gmm-py
for i, (mean, covar, color) in enumerate(zip(means, covariances, colors)):
    v, w = linalg.eigh(covar)
    v = 2.0 * np.sqrt(2.0) * np.sqrt(v)
    u = w[pcs[0]] / linalg.norm(w[pcs[0]])

    # Plot an ellipse to show the Gaussian component
    angle = np.atan2(u[pcs[1]], u[pcs[0]])
    angle = 180.0 * angle / np.pi  # convert to degrees
    ell = mpl.patches.Ellipse(mean, v[pcs[0]], v[pcs[1]], angle=180.0 + angle, color=color, label=f"GMM Category {i}")
    ell.set_clip_box(ax.bbox)
    ell.set_alpha(0.3)
    ax.add_artist(ell)

ax.set_title('Flags on the first two PCs')
ax.set_xlabel(f'PC {pcs[0]+1}')
ax.set_ylabel(f'PC {pcs[1]+1}')
ax.set_facecolor('lightgray')
ax.legend()

fig.savefig('../writeup/res/flag-gmm.pdf')

plt.show()

In [ ]:
gmm_categories = dict()

for flag, cat in zip(flags['Flag(s)'], best_gmm.predict(X)):
    l = gmm_categories.get(int(cat), list())
    l.append(flag)
    gmm_categories[int(cat)] = l

{cat: len(values) for cat, values in gmm_categories.items()}

In [ ]:
from tabulate import tabulate

print(tabulate(gmm_categories, headers=gmm_categories.keys()))

# Combined k-means GMM plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

pcs = [0, 1]
pts = (X @ pca.components_[pcs].T).T.values
# plt.scatter(*pts)

# Plot the points
for i, (x, y) in enumerate(pts.T):
    image = plt.imread(flags['path'][i])
    offset_img = OffsetImage(image, zoom=0.1)
    abb = AnnotationBbox(offset_img, (x, y), xycoords='data', frameon=False, zorder=0)
    ax.add_artist(abb)

ax.update_datalim(pts.T)
ax.autoscale()

# Plot the GMM confidence intervals
means = best_gmm.get_params()['gaussianmixture'].means_
covariances = best_gmm.get_params()['gaussianmixture'].covariances_
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# https://scikit-learn.org/stable/auto_examples/mixture/plot_gmm.html#sphx-glr-auto-examples-mixture-plot-gmm-py
for i, (mean, covar, color) in enumerate(zip(means, covariances, colors)):
    v, w = linalg.eigh(covar)
    v = 2.0 * np.sqrt(2.0) * np.sqrt(v)
    u = w[pcs[0]] / linalg.norm(w[pcs[0]])

    # Plot an ellipse to show the Gaussian component
    angle = np.atan2(u[pcs[1]], u[pcs[0]])
    angle = 180.0 * angle / np.pi  # convert to degrees
    ell = mpl.patches.Ellipse(mean, v[pcs[0]], v[pcs[1]], angle=180.0 + angle, color=color, label=f"GMM Category {i}")
    ell.set_clip_box(ax.bbox)
    ell.set_alpha(0.3)
    ax.add_artist(ell)

# Plot the k-means clusters as a Voronoi diagram
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# plt.scatter(*centers_2d.T, zorder=20, color='r', alpha=0.5)
voronoi_plot_2d(Voronoi(centers_2d), ax=ax)

ax.set_xlim(xlim)
ax.set_ylim(ylim)

# Titles etc
ax.set_title('Flags on the first two PCs')
ax.set_xlabel(f'PC {pcs[0]+1}')
ax.set_ylabel(f'PC {pcs[1]+1}')
ax.set_facecolor('lightgray')
ax.legend()

fig.savefig('../writeup/res/flags-kmeans-gmm.pdf')

plt.show()

# Unique Flags

In [ ]:
from  scipy.spatial.distance import cdist

pca138 = PCA(138).fit(X)
reduced = pca138.transform(X)

distances = cdist(reduced, reduced, metric='euclidean')

no_self_dist = distances.copy()
no_self_dist[np.diag_indices(distances.shape[0])] = np.inf

dist_table = pd.DataFrame(data={'flag-country': Y,
                                'avg_dist': np.mean(distances, axis=1),
                                'min_dist': np.min(no_self_dist, axis=1)},)
dist_table.sort_values(by='min_dist', inplace=True, ascending=False)
dist_table

In [ ]:
dist_table[dist_table['flag-country'] == 'Canada-Canada']